<h3 style="text-align: center;"><span style="text-decoration: underline;">WEB SRAPING - REPOSITORIO DE UNAP</span></h3>
<p><strong>Nombre:</strong> Geraldine Gianella Geronimo Oscanoa (20210836)</p>
<p><strong>Descripci&oacute;n del caso:</strong> Se quiere obtener informaci&oacute;n estructurada de las tesis de pregrado y posgrado de Estad&iacute;stica, Inform&aacute;tica o de carreras similares desarrolladas a nivel nacional, a trav&eacute;s de los repositorios institucionales de las universidades.</p>  



### TESIS PREGRADO DE LA ESCUELA PROFESIONAL INGENIERÍA ESTADÍSTICA E INFORMÁTICA

1. Web Scraping

In [19]:
# Cargando librerías o módulos pertinentes
import requests
from bs4 import BeautifulSoup
import re
import itertools

  Para extraer el NÚMERO TOTAL DE TESIS que se encuentran disponibles del repositorio de la UNAP 
    Se usa como url base: http://repositorio.unap.edu.pe/

In [20]:
# Creando una función que permite extraer todas las urls de las páginas que contienen
# las publicaciones encontradas en erespectivo repositorio
def all_pages(url):
    # Descargar el contenido de la página
    page = requests.get(url)
    # Crear un objeto BeautifulSoup a partir del contenido de la página
    soup = BeautifulSoup(page.content, "html.parser")
    # Encontrar el tag a que contiene el url base de las páginas que contienen las publicaciones
    pages = soup.find_all("a", {"class":"next-page-link"})
    # Extraer el url base de las páginas que contienen las publicaciones
    pages_soup = BeautifulSoup(str(pages), 'html.parser')
    a_tags = pages_soup.find_all('a') # find all anchor elements
    base_href = ["http://repositorio.unap.edu.pe/" + a_tag['href'] for a_tag in a_tags][0]
    # Crear lista de límites inferiores de cada url-página
    string_pages = soup.find_all("p", {"class":"pagination-info"})
    matches = re.findall("\d+", str(string_pages[0]))
    lower_limit_max = (max([int(item) for item in matches]) // 20) * 20
    lower_limits = list(range(0, lower_limit_max + 20, 20))
    # Crear lista de urls de cada una de las páginas 
    hrefs_all = [re.sub(r"offset=\d+", "offset=" + str(pag), base_href) for pag in lower_limits]
    return hrefs_all

In [21]:
# Creando una función que permite extraer todas las urls de la tabla de datos de las
# publicaciones de una determina página
def links_scrapy(url):
    # Descargar el contenido de la página
    page = requests.get(url)
    # Crear un objeto BeautifulSoup a partir del contenido de la página
    soup = BeautifulSoup(page.content, "html.parser")
    # Encontrar el tag padre de las publicaciones que contiene su respectivo url
    publication = soup.find_all("div", {"class":"artifact-title"})
    # Extraer los url de los tags hijos respectivos
    publication_soup = BeautifulSoup(str(publication), 'html.parser')
    a_tags = publication_soup.find_all('a') # find all anchor elements
    hrefs = ["http://repositorio.unap.edu.pe" + a_tag['href'] + "?show=full" for a_tag in a_tags]
    return hrefs

In [22]:
every_page_pre = all_pages("http://tesis.unap.edu.pe/handle/20.500.14082/832/browse?type=dateissued")
publications_links_pre = list(itertools.chain(*[links_scrapy(url) for url in every_page_pre]))
#Guardamos el número total de tesis disponibles para poder sacar datos estadísticos más adelante
total_tesis_pregrado=len(publications_links_pre)

    Para extraer el NÚMERO TOTAL DE TESIS DE LA ESCUELA PROFESIONAL ESTADÍSTICA E INFORMÁTICA que se encuentran disponibles del repositorio de la UNAP 
    Se usa como url base: http://tesis.unap.edu.pe/handle/20.500.14082/857//

In [23]:
def all_pages(url):
    # Descargar el contenido de la página
    page = requests.get(url)
    # Crear un objeto BeautifulSoup a partir del contenido de la página
    soup = BeautifulSoup(page.content, "html.parser")
    # Encontrar el tag a que contiene el url base de las páginas que contienen las publicaciones
    pages = soup.find_all("a", {"class":"next-page-link"})
    # Extraer el url base de las páginas que contienen las publicaciones
    pages_soup = BeautifulSoup(str(pages), 'html.parser')
    a_tags = pages_soup.find_all('a') # find all anchor elements
    base_href = ["http://tesis.unap.edu.pe/handle/20.500.14082/857/" + a_tag['href'] for a_tag in a_tags][0]
    # Crear lista de límites inferiores de cada url-página
    string_pages = soup.find_all("p", {"class":"pagination-info"})
    matches = re.findall("\d+", str(string_pages[0]))
    lower_limit_max = (max([int(item) for item in matches]) // 20) * 20
    lower_limits = list(range(0, lower_limit_max + 20, 20))
    # Crear lista de urls de cada una de las páginas 
    hrefs_all = [re.sub(r"offset=\d+", "offset=" + str(pag), base_href) for pag in lower_limits]
    return hrefs_all

In [24]:
every_page_pre_ei = all_pages("http://tesis.unap.edu.pe/handle/20.500.14082/857/browse?type=dateissued")
publications_links_pre_ei = list(itertools.chain(*[links_scrapy(url) for url in every_page_pre_ei]))
publications_links_pre_ei

['http://repositorio.unap.edu.pe/handle/20.500.14082/7261?show=full',
 'http://repositorio.unap.edu.pe/handle/20.500.14082/7249?show=full',
 'http://repositorio.unap.edu.pe/handle/20.500.14082/13282?show=full',
 'http://repositorio.unap.edu.pe/handle/20.500.14082/4475?show=full',
 'http://repositorio.unap.edu.pe/handle/20.500.14082/7269?show=full',
 'http://repositorio.unap.edu.pe/handle/20.500.14082/2021?show=full',
 'http://repositorio.unap.edu.pe/handle/20.500.14082/7257?show=full',
 'http://repositorio.unap.edu.pe/handle/20.500.14082/3857?show=full',
 'http://repositorio.unap.edu.pe/handle/20.500.14082/7298?show=full',
 'http://repositorio.unap.edu.pe/handle/20.500.14082/7272?show=full',
 'http://repositorio.unap.edu.pe/handle/20.500.14082/7270?show=full',
 'http://repositorio.unap.edu.pe/handle/20.500.14082/7251?show=full',
 'http://repositorio.unap.edu.pe/handle/20.500.14082/5086?show=full',
 'http://repositorio.unap.edu.pe/handle/20.500.14082/5578?show=full',
 'http://repositori

In [25]:
import requests
import pandas as pd
headers = {"User-agent": 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.130 Safari/537.36'}

#Creamos las listas donde guardaremos la información de las tesis
años = []
instituciones = []
titulos = []
autores = []
contribuidores = []
grados = []
resumenes = []
diciplinas = []
niveles= []
level = []
#Añadimos un contador que será de utilidad para colocar un dato vacio en caso la tesis no cuente con la información que se requiere 
n = 0

#
for i in range(len(publications_links_pre_ei)):
    respuesta = requests.get(publications_links_pre_ei[i], headers=headers)
    tabla_tesis = pd.read_html(respuesta.content, encoding = 'utf8')
    tabla_1 = tabla_tesis[0]

#Seleccionamos el dato de la fecha de tesis de cada link y lo añadimos en su respectiva lista
    for j in range(len(tabla_1)):
        if tabla_1.iloc[j,0] == 'dc.date.issued':
            año=tabla_1.iloc[j,1]
            años.append(año[0:4])
            n = n + 1
    if n == 0:
        años.append(" ")

    else:
        n = 0


#Seleccionamos el dato de la institución de cada link y lo añadimos en su respectiva lista
    for j in range(len(tabla_1)):
        if tabla_1.iloc[j,0] == 'dc.publisher':
            institucion=tabla_1.iloc[j,1]
            instituciones.append(institucion)
            n = n + 1
    if n == 0:
        instituciones.append(" ")

    else:
        n = 0


#Seleccionamos el dato del titulo de cada link y lo añadimos en su respectiva lista
    for j in range(len(tabla_1)):        
        if tabla_1.iloc[j,0] == 'dc.title':
            titulo=tabla_1.iloc[j,1]
            titulos.append(titulo)
            n = n + 1
    if n == 0:
            titulos.append(" ")

    else:
        n = 0

#Seleccionamos el dato del autor de tesis de cada link y lo añadimos en su respectiva lista
    for j in range(len(tabla_1)):
        if tabla_1.iloc[j,0] == 'dc.contributor.author':
            autor=tabla_1.iloc[j,1]
            autores.append(autor)
            n = n + 1
#En caso exista más de un autor, sólo seleccionamos el primero
            break
    if n == 0:
            autores.append(" ")

    else:
        n = 0 


#Seleccionamos el dato del grado de tesis de cada link y lo añadimos en su respectiva lista
    for j in range(len(tabla_1)):
        if tabla_1.iloc[j,0] == 'thesis.degree.name':
            grado=tabla_1.iloc[j,1]
            grados.append(grado)
            n = n + 1
    if n == 0:
            grados.append(" ")

    else:
        n = 0
        
    
    
    for j in range(len(tabla_1)):
        if tabla_1.iloc[j,0] == 'thesis.degree.level':
            nivel=tabla_1.iloc[j,1]
            niveles.append(nivel)
            n = n + 1
    if n == 0:
            niveles.append(" ")

    else:
        n = 0


#Seleccionamos el dato del asesor de tesis de cada link y lo añadimos en su respectiva lista
    for j in range(len(tabla_1)):
        if tabla_1.iloc[j,0] == 'dc.contributor.advisor':
            contribuidor=tabla_1.iloc[j,1]
            contribuidores.append(contribuidor)
            n = n + 1
#En caso exista más de un asesor, sólo seleccionamos el primero
            break
    if n == 0:
            contribuidores.append(" ")
    else:
        n = 0


#Seleccionamos el dato del resumen de tesis de cada link y lo añadimos en su respectiva lista
    for j in range(len(tabla_1)):
        if tabla_1.iloc[j,0] == 'dc.description.abstract':
            resumen=tabla_1.iloc[j,1]
            resumenes.append(resumen)
            n = n + 1
    if n == 0:
            resumenes.append(" ")
    else:
        n = 0
    
    
    for j in range(len(tabla_1)):
        if tabla_1.iloc[j,0] == 'thesis.degree.discipline':
            diciplina=tabla_1.iloc[j,1]
            diciplinas.append(diciplina)
            n = n + 1
    if n == 0:
            diciplinas.append(" ")
    else:
        n = 0
    
#Añadimos el dato de 'pregrado' en cada tesis
    level.append("pregrado")



#print('Lista de años: ',años)
#print('Lista de instituciones: ',instituciones)
#print('Lista de titulos: ',titulos)
#print('Lista de autores: ',autores)
#print('Lista de contribuidores: ',contribuidores)
#print('Lista de grados: ',grados)
#print('Lista de resumenes: ',resumenes)

In [26]:
#Creamos un df propio donde almacenaremos la información de las tesis de estadística informática en nivel pregrado
df_pre1 = pd.DataFrame()

df_pre1['instituciones'] = instituciones
df_pre1['titulos'] = titulos
df_pre1['autores'] = autores
df_pre1['contribuidores'] = contribuidores
df_pre1['resumenes'] = resumenes
df_pre1['grados'] = grados
df_pre1['niveles']=niveles
df_pre1['años'] = años
df_pre1['diciplinas']=diciplinas
df_pre1['level']=level


 Para extraer el nNÚMERO TOTAL DE TESIS DE LA ESCUELA PROFESIONAL INGENIERÍA DE SISTEMAS que se encuentran disponibles del repositorio de la UNAP 
    Se usa como url base: http://tesis.unap.edu.pe/handle/20.500.14082/849/

In [27]:
def all_pages(url):
    # Descargar el contenido de la página
    page = requests.get(url)
    # Crear un objeto BeautifulSoup a partir del contenido de la página
    soup = BeautifulSoup(page.content, "html.parser")
    # Encontrar el tag a que contiene el url base de las páginas que contienen las publicaciones
    pages = soup.find_all("a", {"class":"next-page-link"})
    # Extraer el url base de las páginas que contienen las publicaciones
    pages_soup = BeautifulSoup(str(pages), 'html.parser')
    a_tags = pages_soup.find_all('a') # find all anchor elements
    base_href = ["http://tesis.unap.edu.pe/handle/20.500.14082/849/" + a_tag['href'] for a_tag in a_tags][0]
    # Crear lista de límites inferiores de cada url-página
    string_pages = soup.find_all("p", {"class":"pagination-info"})
    matches = re.findall("\d+", str(string_pages[0]))
    lower_limit_max = (max([int(item) for item in matches]) // 20) * 20
    lower_limits = list(range(0, lower_limit_max + 20, 20))
    # Crear lista de urls de cada una de las páginas 
    hrefs_all = [re.sub(r"offset=\d+", "offset=" + str(pag), base_href) for pag in lower_limits]
    return hrefs_all

In [28]:
every_page_pre_is = all_pages("http://tesis.unap.edu.pe/handle/20.500.14082/849/browse?type=dateissued")
publications_links_pre_is = list(itertools.chain(*[links_scrapy(url) for url in every_page_pre_is]))
publications_links_pre_is

['http://repositorio.unap.edu.pe/handle/20.500.14082/9596?show=full',
 'http://repositorio.unap.edu.pe/handle/20.500.14082/2368?show=full',
 'http://repositorio.unap.edu.pe/handle/20.500.14082/2689?show=full',
 'http://repositorio.unap.edu.pe/handle/20.500.14082/2242?show=full',
 'http://repositorio.unap.edu.pe/handle/20.500.14082/1913?show=full',
 'http://repositorio.unap.edu.pe/handle/20.500.14082/2374?show=full',
 'http://repositorio.unap.edu.pe/handle/20.500.14082/1811?show=full',
 'http://repositorio.unap.edu.pe/handle/20.500.14082/2817?show=full',
 'http://repositorio.unap.edu.pe/handle/20.500.14082/1792?show=full',
 'http://repositorio.unap.edu.pe/handle/20.500.14082/1841?show=full',
 'http://repositorio.unap.edu.pe/handle/20.500.14082/1853?show=full',
 'http://repositorio.unap.edu.pe/handle/20.500.14082/2514?show=full',
 'http://repositorio.unap.edu.pe/handle/20.500.14082/1830?show=full',
 'http://repositorio.unap.edu.pe/handle/20.500.14082/2718?show=full',
 'http://repositorio

In [29]:
import requests
import pandas as pd
headers = {"User-agent": 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.130 Safari/537.36'}

#Creamos las listas donde guardaremos la información de las tesis
años = []
instituciones = []
titulos = []
autores = []
contribuidores = []
grados = []
resumenes = []
diciplinas = []
niveles= []
level=[]
#Añadimos un contador que será de utilidad para colocar un dato vacio en caso la tesis no cuente con la información que se requiere 
n = 0

#
for i in range(len(publications_links_pre_is)):
    respuesta = requests.get(publications_links_pre_is[i], headers=headers)
    tabla_tesis = pd.read_html(respuesta.content, encoding = 'utf8')
    tabla_1 = tabla_tesis[0]

#Seleccionamos el dato de la fecha de tesis de cada link y lo añadimos en su respectiva lista
    for j in range(len(tabla_1)):
        if tabla_1.iloc[j,0] == 'dc.date.issued':
            año=tabla_1.iloc[j,1]
            años.append(año[0:4])
            n = n + 1
    if n == 0:
        años.append(" ")

    else:
        n = 0


#Seleccionamos el dato de la institución de cada link y lo añadimos en su respectiva lista
    for j in range(len(tabla_1)):
        if tabla_1.iloc[j,0] == 'dc.publisher':
            institucion=tabla_1.iloc[j,1]
            instituciones.append(institucion)
            n = n + 1
    if n == 0:
        instituciones.append(" ")

    else:
        n = 0


#Seleccionamos el dato del titulo de cada link y lo añadimos en su respectiva lista
    for j in range(len(tabla_1)):        
        if tabla_1.iloc[j,0] == 'dc.title':
            titulo=tabla_1.iloc[j,1]
            titulos.append(titulo)
            n = n + 1
    if n == 0:
            titulos.append(" ")

    else:
        n = 0

#Seleccionamos el dato del autor de tesis de cada link y lo añadimos en su respectiva lista
    for j in range(len(tabla_1)):
        if tabla_1.iloc[j,0] == 'dc.contributor.author':
            autor=tabla_1.iloc[j,1]
            autores.append(autor)
            n = n + 1
#En caso exista más de un autor, sólo seleccionamos el primero
            break
    if n == 0:
            autores.append(" ")

    else:
        n = 0 


#Seleccionamos el dato del grado de tesis de cada link y lo añadimos en su respectiva lista
    for j in range(len(tabla_1)):
        if tabla_1.iloc[j,0] == 'thesis.degree.name':
            grado=tabla_1.iloc[j,1]
            grados.append(grado)
            n = n + 1
    if n == 0:
            grados.append(" ")

    else:
        n = 0
        
    
    
    for j in range(len(tabla_1)):
        if tabla_1.iloc[j,0] == 'thesis.degree.level':
            nivel=tabla_1.iloc[j,1]
            niveles.append(nivel)
            n = n + 1
    if n == 0:
            niveles.append(" ")

    else:
        n = 0


#Seleccionamos el dato del asesor de tesis de cada link y lo añadimos en su respectiva lista
    for j in range(len(tabla_1)):
        if tabla_1.iloc[j,0] == 'dc.contributor.advisor':
            contribuidor=tabla_1.iloc[j,1]
            contribuidores.append(contribuidor)
            n = n + 1
#En caso exista más de un asesor, sólo seleccionamos el primero
            break
    if n == 0:
            contribuidores.append(" ")
    else:
        n = 0


#Seleccionamos el dato del resumen de tesis de cada link y lo añadimos en su respectiva lista
    for j in range(len(tabla_1)):
        if tabla_1.iloc[j,0] == 'dc.description.abstract':
            resumen=tabla_1.iloc[j,1]
            resumenes.append(resumen)
            n = n + 1
    if n == 0:
            resumenes.append(" ")
    else:
        n = 0
    
    
    for j in range(len(tabla_1)):
        if tabla_1.iloc[j,0] == 'thesis.degree.discipline':
            diciplina=tabla_1.iloc[j,1]
            diciplinas.append(diciplina)
            n = n + 1
    if n == 0:
            diciplinas.append(" ")
    else:
        n = 0

#Añadimos el dato de 'pregrado' en cada tesis
    level.append("pregrado")



#print('Lista de años: ',años)
#print('Lista de instituciones: ',instituciones)
#print('Lista de titulos: ',titulos)
#print('Lista de autores: ',autores)
#print('Lista de contribuidores: ',contribuidores)
#print('Lista de grados: ',grados)
#print('Lista de resumenes: ',resumenes)

In [30]:
#Creamos un df propio donde almacenaremos la información de las tesis de Ingeniería de Sistemas en nivel pregrado
df_pre2 = pd.DataFrame()

df_pre2['instituciones'] = instituciones
df_pre2['titulos'] = titulos
df_pre2['autores'] = autores
df_pre2['contribuidores'] = contribuidores
df_pre2['resumenes'] = resumenes
df_pre2['grados'] = grados
df_pre2['niveles']=niveles
df_pre2['años'] = años
df_pre2['diciplinas']=diciplinas
df_pre2['level']=level

In [31]:
#Unimos los 2 df de ambas carreras
df_pre_final = pd.DataFrame(df_pre1.append(df_pre2, 
                  ignore_index = True))

C:\Users\Geraldine\AppData\Local\Temp\ipykernel_4068\1673346613.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_pre_final = pd.DataFrame(df_pre1.append(df_pre2,


In [32]:
df_pre_final

,instituciones,titulos,autores,contribuidores,resumenes,grados,niveles,años,diciplinas,level
0,Universidad Nacional del Altiplano,Análisis y caracterización de los factores de ...,"Masias Gutierrez, Luis Alberto","Paredes Quispe, Juan Reynaldo",Los profesionales de salud que laboran en el H...,Ingeniero Estadístico e Informático,Título Profesional,2006,Ingeniería Estadística e Informática,pregrado
1,Universidad Nacional del Altiplano,Desarrollo del sistema de información multiusu...,"Auquitias Condori, Gladys Marleny","Quispe Mamani, Godofredo","El presente trabajo de investigación, se desar...",Ingeniero Estadístico e Informático,Título Profesional,2007,Ingeniería Estadística e Informática,pregrado
2,Universidad Nacional del Altiplano,"Deficit habitacional de viviendas, por provinc...","Mamani Quispe, Jorge Luis","Azañero de Aguirre, Emma Orfelinda",El presente trabajo de investigación se desarr...,Ingeniero Estadístico e Informático,Título Profesional,2007,Ingeniería Estadística e Informática,pregrado
3,Universidad Nacional del Altiplano,Software de consulta virtual para la atención ...,"Galarza Lerma, Roxana","Apaza Tarqui, Alejadro",Las bibliotecas especializadas poseen coleccio...,Ingeniero Estadístico e Informático,Título Profesional,2009,Ingeniería Estadística e Informática,pregrado
4,Universidad Nacional del Altiplano,Factores educativos que determinan el buen des...,"Luna Mansilla, Catherine","Mendoza Mollocondo, Charles Ignacio",El Ministerio de Educación en coordinación con...,Ingeniero Estadístico e Informático,Título Profesional,2010,Ingeniería Estadística e Informática,pregrado
...,...,...,...,...,...,...,...,...,...,...
333,Universidad Nacional del Altiplano,Sistema de información web para mejorar la ges...,"Ponce Castillo, Alexander Antonio","Calderon Vilca, Edwin Fredy",La presente investigación titulada “Sistema de...,Ingeniero de Sistemas,,2022,Ingeniería de Sistemas,pregrado
334,Universidad Nacional del Altiplano,Modelo predictivo aplicando análisis de sentim...,"Chura Flores, Ernesto Zhildeer","Condori Alejo, Henry Iván",Hoy en día el Bitcoin es la criptodivisa con m...,Ingeniero de Sistemas,,2022,Ingeniería de Sistemas,pregrado
335,Universidad Nacional del Altiplano,Desarrollo de una aplicación multiplataforma c...,"Ichuta Arias, Caleb Fernando","Sotomayor Alzamora, Guina Guadalupe",El aprendizaje de la lengua aimara se originó ...,Ingeniero de Sistemas,,2022,Ingeniería de Sistemas,pregrado
336,Universidad Nacional del Altiplano,Modelo de implementación VLAN en la red de áre...,"Ccari Calcina, Richard Elmer","Flores Velásquez, Edelfré",La presente investigación titulada Modelo de i...,Ingeniero de Sistemas,,2023,Ingeniería de Sistemas,pregrado


#### Conclusiones:

In [33]:
print("-El porcentaje de tesis en nivel PREGRADO de la carrera Estadística Informática es: ", (len(publications_links_ei)/total_tesis_pregrado)*100, "%")
print("-El porcentaje de tesis en nivel PREGRADO de la carrera Ingeniería de Sistemas es: ", (len(publications_links_is)/total_tesis_pregrado)*100, "%")
print("-Respecto al año de publicación de las tesis:")
df_pre_final['años'].describe()

-El porcentaje de tesis en nivel PREGRADO de la carrera Estadística Informática es:  6.8248175182481745 %
-El porcentaje de tesis en nivel PREGRADO de la carrera Ingeniería de Sistemas es:  5.510948905109489 %
-Respecto al año de publicación de las tesis:


count      338
unique      17
top       2017
freq        63
Name: años, dtype: object

#### TESIS POSGRADO DE LA ESCUELA PROFESIONAL INGENIERÍA ESTADÍSTICA E INFORMÁTICA

In [34]:
def all_pages(url):
    # Descargar el contenido de la página
    page = requests.get(url)
    # Crear un objeto BeautifulSoup a partir del contenido de la página
    soup = BeautifulSoup(page.content, "html.parser")
    # Encontrar el tag a que contiene el url base de las páginas que contienen las publicaciones
    pages = soup.find_all("a", {"class":"next-page-link"})
    # Extraer el url base de las páginas que contienen las publicaciones
    pages_soup = BeautifulSoup(str(pages), 'html.parser')
    a_tags = pages_soup.find_all('a') # find all anchor elements
    base_href = ["http://tesis.unap.edu.pe/handle/20.500.14082/153/" + a_tag['href'] for a_tag in a_tags][0]
    # Crear lista de límites inferiores de cada url-página
    string_pages = soup.find_all("p", {"class":"pagination-info"})
    matches = re.findall("\d+", str(string_pages[0]))
    lower_limit_max = (max([int(item) for item in matches]) // 20) * 20
    lower_limits = list(range(0, lower_limit_max + 20, 20))
    # Crear lista de urls de cada una de las páginas 
    hrefs_all = [re.sub(r"offset=\d+", "offset=" + str(pag), base_href) for pag in lower_limits]
    return hrefs_all

In [35]:
every_page_pos = all_pages("http://tesis.unap.edu.pe/handle/20.500.14082/153/browse?type=dateissued")
publications_links_pos = list(itertools.chain(*[links_scrapy(url) for url in every_page_pos]))
total_tesis_posgrado=len(publications_links_pos)

Para extraer el NÚMERO TOTAL DE TESIS POSGRADO - DOCTORADO que se encuentran disponibles del repositorio de la UNAP 
    Se usa como url base: "http://tesis.unap.edu.pe/handle/20.500.14082/3730/

In [36]:
def all_pages(url):
    # Descargar el contenido de la página
    page = requests.get(url)
    # Crear un objeto BeautifulSoup a partir del contenido de la página
    soup = BeautifulSoup(page.content, "html.parser")
    # Encontrar el tag a que contiene el url base de las páginas que contienen las publicaciones
    pages = soup.find_all("a", {"class":"next-page-link"})
    # Extraer el url base de las páginas que contienen las publicaciones
    pages_soup = BeautifulSoup(str(pages), 'html.parser')
    a_tags = pages_soup.find_all('a') # find all anchor elements
    base_href = ["http://tesis.unap.edu.pe/handle/20.500.14082/3730/" + a_tag['href'] for a_tag in a_tags][0]
    # Crear lista de límites inferiores de cada url-página
    string_pages = soup.find_all("p", {"class":"pagination-info"})
    matches = re.findall("\d+", str(string_pages[0]))
    lower_limit_max = (max([int(item) for item in matches]) // 20) * 20
    lower_limits = list(range(0, lower_limit_max + 20, 20))
    # Crear lista de urls de cada una de las páginas 
    hrefs_all = [re.sub(r"offset=\d+", "offset=" + str(pag), base_href) for pag in lower_limits]
    return hrefs_all

In [37]:
every_page_pos_ie = all_pages("http://tesis.unap.edu.pe/handle/20.500.14082/153/browse?type=dateissued")
publications_links_pos_ie = list(itertools.chain(*[links_scrapy(url) for url in every_page_pos_ie]))
publications_links_pos_ie

['http://repositorio.unap.edu.pe/handle/20.500.14082/258?show=full',
 'http://repositorio.unap.edu.pe/handle/20.500.14082/236?show=full',
 'http://repositorio.unap.edu.pe/handle/20.500.14082/219?show=full',
 'http://repositorio.unap.edu.pe/handle/20.500.14082/235?show=full',
 'http://repositorio.unap.edu.pe/handle/20.500.14082/221?show=full',
 'http://repositorio.unap.edu.pe/handle/20.500.14082/233?show=full',
 'http://repositorio.unap.edu.pe/handle/20.500.14082/224?show=full',
 'http://repositorio.unap.edu.pe/handle/20.500.14082/237?show=full',
 'http://repositorio.unap.edu.pe/handle/20.500.14082/240?show=full',
 'http://repositorio.unap.edu.pe/handle/20.500.14082/230?show=full',
 'http://repositorio.unap.edu.pe/handle/20.500.14082/247?show=full',
 'http://repositorio.unap.edu.pe/handle/20.500.14082/229?show=full',
 'http://repositorio.unap.edu.pe/handle/20.500.14082/231?show=full',
 'http://repositorio.unap.edu.pe/handle/20.500.14082/239?show=full',
 'http://repositorio.unap.edu.pe/h

In [38]:
import requests
import pandas as pd
headers = {"User-agent": 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.130 Safari/537.36'}

#Creamos las listas donde guardaremos la información de las tesis
años = []
instituciones = []
titulos = []
autores = []
contribuidores = []
grados = []
resumenes = []
diciplinas = []
niveles= []
level=[]
#Añadimos un contador que será de utilidad para colocar un dato vacio en caso la tesis no cuente con la información que se requiere 
n = 0

#
for i in range(len(publications_links_pos_ie)):
    respuesta = requests.get(publications_links_pos_ie[i], headers=headers)
    tabla_tesis = pd.read_html(respuesta.content, encoding = 'utf8')
    tabla_1 = tabla_tesis[0]

#Seleccionamos el dato de la fecha de tesis de cada link y lo añadimos en su respectiva lista
    for j in range(len(tabla_1)):
        if tabla_1.iloc[j,0] == 'dc.date.issued':
            año=tabla_1.iloc[j,1]
            años.append(año[0:4])
            n = n + 1
    if n == 0:
        años.append(" ")

    else:
        n = 0


#Seleccionamos el dato de la institución de cada link y lo añadimos en su respectiva lista
    for j in range(len(tabla_1)):
        if tabla_1.iloc[j,0] == 'dc.publisher':
            institucion=tabla_1.iloc[j,1]
            instituciones.append(institucion)
            n = n + 1
    if n == 0:
        instituciones.append(" ")

    else:
        n = 0


#Seleccionamos el dato del titulo de cada link y lo añadimos en su respectiva lista
    for j in range(len(tabla_1)):        
        if tabla_1.iloc[j,0] == 'dc.title':
            titulo=tabla_1.iloc[j,1]
            titulos.append(titulo)
            n = n + 1
    if n == 0:
            titulos.append(" ")

    else:
        n = 0

#Seleccionamos el dato del autor de tesis de cada link y lo añadimos en su respectiva lista
    for j in range(len(tabla_1)):
        if tabla_1.iloc[j,0] == 'dc.contributor.author':
            autor=tabla_1.iloc[j,1]
            autores.append(autor)
            n = n + 1
#En caso exista más de un autor, sólo seleccionamos el primero
            break
    if n == 0:
            autores.append(" ")

    else:
        n = 0 


#Seleccionamos el dato del grado de tesis de cada link y lo añadimos en su respectiva lista
    for j in range(len(tabla_1)):
        if tabla_1.iloc[j,0] == 'thesis.degree.name':
            grado=tabla_1.iloc[j,1]
            grados.append(grado)
            n = n + 1
    if n == 0:
            grados.append(" ")

    else:
        n = 0
        
    
    
    for j in range(len(tabla_1)):
        if tabla_1.iloc[j,0] == 'thesis.degree.level':
            nivel=tabla_1.iloc[j,1]
            niveles.append(nivel)
            n = n + 1
    if n == 0:
            niveles.append(" ")

    else:
        n = 0


#Seleccionamos el dato del asesor de tesis de cada link y lo añadimos en su respectiva lista
    for j in range(len(tabla_1)):
        if tabla_1.iloc[j,0] == 'dc.contributor.advisor':
            contribuidor=tabla_1.iloc[j,1]
            contribuidores.append(contribuidor)
            n = n + 1
#En caso exista más de un asesor, sólo seleccionamos el primero
            break
    if n == 0:
            contribuidores.append(" ")
    else:
        n = 0


#Seleccionamos el dato del resumen de tesis de cada link y lo añadimos en su respectiva lista
    for j in range(len(tabla_1)):
        if tabla_1.iloc[j,0] == 'dc.description.abstract':
            resumen=tabla_1.iloc[j,1]
            resumenes.append(resumen)
            n = n + 1
    if n == 0:
            resumenes.append(" ")
    else:
        n = 0
    
    
    for j in range(len(tabla_1)):
        if tabla_1.iloc[j,0] == 'thesis.degree.discipline':
            diciplina=tabla_1.iloc[j,1]
            diciplinas.append(diciplina)
            n = n + 1
    if n == 0:
            diciplinas.append(" ")
    else:
        n = 0

#Añadimos el dato de 'posgrado' en cada tesis
    level.append("posgrado")



#print('Lista de años: ',años)
#print('Lista de instituciones: ',instituciones)
#print('Lista de titulos: ',titulos)
#print('Lista de autores: ',autores)
#print('Lista de contribuidores: ',contribuidores)
#print('Lista de grados: ',grados)
#print('Lista de resumenes: ',resumenes)

In [39]:
#Creamos un df propio donde almacenaremos la información nivel posgrado
df_pos = pd.DataFrame()

df_pos['instituciones'] = instituciones
df_pos['titulos'] = titulos
df_pos['autores'] = autores
df_pos['contribuidores'] = contribuidores
df_pos['resumenes'] = resumenes
df_pos['grados'] = grados
df_pos['niveles']=niveles
df_pos['años'] = años
df_pos['diciplinas']=diciplinas
df_pos['level']=level

In [40]:
df_pos

,instituciones,titulos,autores,contribuidores,resumenes,grados,niveles,años,diciplinas,level
0,Universidad Nacional del Altiplano,Curriculum por competencias y desempeño labora...,"Coyla Zela, Mario Aurelio","Vizcarra Herles, Nina Leonor",El trabajo de investigación tiene el propósito...,Doctor Scientiae en Educación,Doctorado,2008,Educación,posgrado
1,Universidad Nacional del Altiplano,Las metodologías pedagógicas colectivas y el d...,"Mamani Aguilar, Oscar","Gutiérrez Alberoni, José Dante","El trabajo de Tesis Doctoral en Educación, tit...",Doctor Scientiae en Educación,Doctorado,2009,Educación,posgrado
2,Universidad Nacional del Altiplano,Costos gerenciales y su influencia en la calid...,"Collantes Menis, Obdulio","Luque Mamani, Juan",El contenido del trabajo de investigación titu...,Doctor Scientiae en Contabilidad y Administración,Doctorado,2009,Contabilidad y Administración,posgrado
3,Universidad Nacional del Altiplano,La autoria mediata: fuentes y perspectivas fre...,"Galvez Condori, Walter Salvador","Espezúa Salmón, Boris Gilmar",Dentro del desarrollo de las actividades progr...,Doctor Scientiae en Derecho,Doctorado,2010,Derecho,posgrado
4,Universidad Nacional del Altiplano,La corrupción institucionalizada y su relación...,"Aliaga Ortega, Wenceslao","Valdivia Yábar, Silvia Verónica",El objetivo de la investigación fue analizar l...,Doctor Scientiae en Contabilidad y Administración,Doctorado,2010,Contabilidad y Administración,posgrado
...,...,...,...,...,...,...,...,...,...,...
415,Universidad Nacional del Altiplano,Deber de protección del estado respecto a pers...,"Isia Larico, Sonia Marizela","Casazola Ccama, Juan",La investigación tiene como objetivo analizar ...,Doctoris Scientiae en Derecho,,2022,Derecho,posgrado
416,Universidad Nacional del Altiplano. Repositori...,Características tecnológicas y densidad folicu...,"Machaca Machaca, Virgilio","Apaza Zuñiga, Edgar",El vellón de la llama (Lama glama) posee doble...,"Doctor en Ciencia, Tecnología y Medio Ambiente",,2022,"Ciencia, Tecnología y Medio Ambiente",posgrado
417,Universidad Nacional del Altiplano. Repositori...,Imputation of missing data in photovoltaic pan...,"Huaquipaco Encinas, Saul",Carpio Vargas Edgar Eloy,"In scientific research, data acquisition and p...",Doctor en Ciencias de la Ingeniería Mecánica E...,,2022,Ciencias de la Ingeniería Mecánica Eléctrica,posgrado
418,Universidad Nacional del Altiplano. Repositori...,Relación de la calidad de las aguas subterráne...,"Millones Chafloque, Audberto","Quispe German, Belizario","En los últimos diez años, el déficit de aguas ...","Doctor en Ciencia, Tecnología y Medio Ambiente",,2022,"Ciencia, Tecnología y Medio Ambiente",posgrado


In [41]:
carreras_e={i:diciplinas.count(i) for i in diciplinas}
df_posgrado = pd.DataFrame([[key, carreras_e[key]] for key in carreras_e.keys()], columns=['Grado', 'Count'])
df_posgrado

,Grado,Count
0,Educación,50
1,Contabilidad y Administración,28
2,Derecho,31
3,Ciencias Sociales,26
4,Economía y Gestión,11
5,"Ciencia, Tecnología y Medio Ambiente",132
6,Ciencias de la Salud,41
7,Administración,23
8,Ciencias de la Computación,18
9,Economía y Políticas Públicas,14


Teniendo en cuenta la información obetnida, los únicos grados de nuestro interés son: "Estadística Aplicada" y "Estadística Informática"  
Podemos sacar las siguientes conclusiones:

#### Conclusiones:

In [42]:
print("-El porcentaje de tesis en nivel POSGRADO-DOCTORADO de  Estadística Informática es: ", (df_posgrado.iloc[13][1]/total_tesis_posgrado)*100, "%")
print("-El porcentaje de tesis en nivel POSGRADO-DOCTORADO de  Estadística Aplicada: ", (df_posgrado.iloc[18][1]/total_tesis_posgrado)*100, "%")

-El porcentaje de tesis en nivel POSGRADO-DOCTORADO de  Estadística Informática es:  0.5720823798627002 %
-El porcentaje de tesis en nivel POSGRADO-DOCTORADO de  Estadística Aplicada:  0.07627765064836003 %


In [43]:
#Unimos los df de ambos niveles
df_final = pd.DataFrame(df_pre_final.append(df_pos, 
                  ignore_index = True))

C:\Users\Geraldine\AppData\Local\Temp\ipykernel_4068\3257773961.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_final = pd.DataFrame(df_pre_final.append(df_pos,


In [44]:
df_final

,instituciones,titulos,autores,contribuidores,resumenes,grados,niveles,años,diciplinas,level
0,Universidad Nacional del Altiplano,Análisis y caracterización de los factores de ...,"Masias Gutierrez, Luis Alberto","Paredes Quispe, Juan Reynaldo",Los profesionales de salud que laboran en el H...,Ingeniero Estadístico e Informático,Título Profesional,2006,Ingeniería Estadística e Informática,pregrado
1,Universidad Nacional del Altiplano,Desarrollo del sistema de información multiusu...,"Auquitias Condori, Gladys Marleny","Quispe Mamani, Godofredo","El presente trabajo de investigación, se desar...",Ingeniero Estadístico e Informático,Título Profesional,2007,Ingeniería Estadística e Informática,pregrado
2,Universidad Nacional del Altiplano,"Deficit habitacional de viviendas, por provinc...","Mamani Quispe, Jorge Luis","Azañero de Aguirre, Emma Orfelinda",El presente trabajo de investigación se desarr...,Ingeniero Estadístico e Informático,Título Profesional,2007,Ingeniería Estadística e Informática,pregrado
3,Universidad Nacional del Altiplano,Software de consulta virtual para la atención ...,"Galarza Lerma, Roxana","Apaza Tarqui, Alejadro",Las bibliotecas especializadas poseen coleccio...,Ingeniero Estadístico e Informático,Título Profesional,2009,Ingeniería Estadística e Informática,pregrado
4,Universidad Nacional del Altiplano,Factores educativos que determinan el buen des...,"Luna Mansilla, Catherine","Mendoza Mollocondo, Charles Ignacio",El Ministerio de Educación en coordinación con...,Ingeniero Estadístico e Informático,Título Profesional,2010,Ingeniería Estadística e Informática,pregrado
...,...,...,...,...,...,...,...,...,...,...
753,Universidad Nacional del Altiplano,Deber de protección del estado respecto a pers...,"Isia Larico, Sonia Marizela","Casazola Ccama, Juan",La investigación tiene como objetivo analizar ...,Doctoris Scientiae en Derecho,,2022,Derecho,posgrado
754,Universidad Nacional del Altiplano. Repositori...,Características tecnológicas y densidad folicu...,"Machaca Machaca, Virgilio","Apaza Zuñiga, Edgar",El vellón de la llama (Lama glama) posee doble...,"Doctor en Ciencia, Tecnología y Medio Ambiente",,2022,"Ciencia, Tecnología y Medio Ambiente",posgrado
755,Universidad Nacional del Altiplano. Repositori...,Imputation of missing data in photovoltaic pan...,"Huaquipaco Encinas, Saul",Carpio Vargas Edgar Eloy,"In scientific research, data acquisition and p...",Doctor en Ciencias de la Ingeniería Mecánica E...,,2022,Ciencias de la Ingeniería Mecánica Eléctrica,posgrado
756,Universidad Nacional del Altiplano. Repositori...,Relación de la calidad de las aguas subterráne...,"Millones Chafloque, Audberto","Quispe German, Belizario","En los últimos diez años, el déficit de aguas ...","Doctor en Ciencia, Tecnología y Medio Ambiente",,2022,"Ciencia, Tecnología y Medio Ambiente",posgrado


In [45]:
#Exportamos el df final
df_final.to_csv('UNA.csv', index=False)